# Wind turbine analysis

This notebook is a simple analysis of the wind turbine data [provided by the U.S. Geological Survey (USGS)](https://eerscmap.usgs.gov/uswtdb/data/) for the San Antonio Express-News.

Questions we have for the data:
- How many turbines currently operate across the country?
- How many currently operate in Texas?
- Which Texas counties have the most turbines?
- Which wind farms are the largest in Texas?

Before we do anything, let's import the libraries we'll need.

In [1]:
# We install pandas for data analysis
import pandas as pd

# We install itables for interactive tables
from itables import show

Now we import the data we collected from the USGS. Definitions of the columns are available [here](https://eerscmap.usgs.gov/uswtdb/api-doc/#keyValue) ([Web Archive](https://web.archive.org/web/20220917234036/https://eerscmap.usgs.gov/uswtdb/api-doc/)).

In [2]:
full_df = pd.read_csv('data/uswtdb_v5_3_20230113.csv')

show(full_df)

Alright, there are 72,669 turbines operating across the country. Let's see how many are in each state.

I'm importing an additional dataframe that contains the full names of states to merge into our pivot table for readability.

In [3]:
state_abbrev_df = pd.read_csv('data/state_abbreviations.csv')

# Create a pivot_table that counts the number of case_ids for each t_state. We also sort the values in descending order.
state_t_count_df = pd.pivot_table(full_df, values='case_id', index=['t_state'], aggfunc='count').sort_values(by='case_id', ascending=False)

# Merge the state_t_count_df with the state_abbrev_df, just to improve readability.
state_t_count_df = pd.merge(state_t_count_df, state_abbrev_df, left_on='t_state', right_on='abbreviations')

# We don't need the abbreviations column anymore, so we drop it.
state_t_count_df = state_t_count_df.drop(columns=['abbreviations'])

# Here I'm renaming the columns, again just to improve readability..
state_t_count_df = state_t_count_df.rename(columns={'t_state': 'State', 'case_id': 'Turbine count'})

# Here I'm defining the names of the columns.
state_t_count_df.columns = ['Turbine count', 'State']

# Reorder the columns
state_t_count_df = state_t_count_df[['State', 'Turbine count']]

# Show the pivot_table
show(state_t_count_df)

Alright, Texas has more wind turbines than any other state and it's not close.

Texas has more than three times as many turbines as California, which is third.

Now let's isolate the turbines in Texas and see which counties have the most.

In [4]:
# Let's create a new dataframe with turbines in the state of Texas. The column we filter is 't_state'
texas_df = full_df[full_df['t_state'] == 'TX']

show(texas_df)

# I'm interested in creating an animated map in Flourish of all the Texas wind turbines coming online. I can't have any rows with missing values in the 'p_year' column, so I'm filtering those out.
animated_map_df = texas_df[texas_df['p_year'].notna()]

# Convert p_year to datetime
animated_map_df['p_year'] = pd.to_datetime(animated_map_df['p_year'], format='%Y')

# Here I'm exporting the data to a csv file, so I can use it in Flourish.
animated_map_df.to_csv('output/texas_wind_turbines.csv', index=False)

/var/folders/w4/djgt59l94t51k1198f2hnn980000gn/T/ipykernel_12551/3422785734.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  animated_map_df['p_year'] = pd.to_datetime(animated_map_df['p_year'], format='%Y')


When it comes to finding the largest wind farms, we have a few options. We could look at the area covered by the turbines, the number of turbines, or the total capacity of the turbines.

For this analysis I'm going to focus on the number of turbines. [The story](https://www.expressnews.com/news/texas/article/new-wind-turbines-west-texas-17822375.php) that inspired this quick-turn project was about the visual clutter wind turbines can create — not their total electrical generation capacity.

First let's find the Texas counties with the most turbines.

In [5]:
# We create a pivot table that counts every wind turbine in each county. There is a unique case_id for each turbine. We sort the values in descending order.
turbine_county_count = pd.pivot_table(texas_df, values='case_id', index=['t_county'], aggfunc='count').sort_values(by='case_id', ascending=False)

# Reset the index
turbine_county_count = turbine_county_count.reset_index()

# Rename the columns
turbine_county_count = turbine_county_count.rename(columns={'t_county': 'County', 'case_id': 'Turbine count'})

# For each value in the County column, do a find and replace to remove the word "County". We do this so that Datawrapper understands the county names.
turbine_county_count['County'] = turbine_county_count['County'].str.replace(' County', '')

# Export the data to a csv file
turbine_county_count.to_csv('output/turbine_county_count.csv', index=False)

show(turbine_county_count)

There's a clear winner here. Nolan County has 1,411 turbines, which is more than double the next county.

Now let's find which wind farms have the most turbines. I'm also going to merge in the county names to make it easier for me to see whether any come from Nolan County.

In [9]:
# Create a pivot_table that shows the number of turbines per p_name (which is the name of the wind project)
wind_farm_t_count_pivot = pd.pivot_table(texas_df, values='case_id', index=['p_name'], aggfunc='count').sort_values(by='case_id', ascending=False)

# We don't want any wind projects with "unknown" in the name
wind_farm_t_count_pivot = wind_farm_t_count_pivot[~wind_farm_t_count_pivot.index.str.contains('unknown', case=False)]

# Rename wind_farm_t_count_pivot case_id column to turbine_count
wind_farm_t_count_pivot = wind_farm_t_count_pivot.rename(columns={'case_id': 'turbine_count'})

# Create a new dataframe that dedupes the texas_df dataframe by p_name
deduped_texas_df = texas_df.drop_duplicates(subset=['p_name'])

# Merge the deduped_texas_df with the wind_farm_t_count_pivot
wind_farm_t_count_pivot = pd.merge(deduped_texas_df, wind_farm_t_count_pivot, left_on='p_name', right_on='p_name').sort_values(by='turbine_count', ascending=False)

# Drop everything but p_name, turbine_count and t_county
wind_farm_t_count_pivot = wind_farm_t_count_pivot[['p_name', 'turbine_count', 't_county']]

# Reset the index
wind_farm_t_count_pivot = wind_farm_t_count_pivot.reset_index(drop=True)

# Rename p_name to Wind Farm, turbine_count to Turbine Count, and t_county to County
wind_farm_t_count_pivot = wind_farm_t_count_pivot.rename(columns={'p_name': 'Wind Farm', 'turbine_count': 'Turbine Count', 't_county': 'County'})

# Reorder the columns
wind_farm_t_count_pivot = wind_farm_t_count_pivot[['Wind Farm', 'County', 'Turbine Count']]

show(wind_farm_t_count_pivot)

In [7]:
# Filter texas_df to only include turbines on "Firewheel Wind" us the p_name column
firewheel_wind_df = texas_df[texas_df['p_name'] == 'Firewheel Wind']

show(firewheel_wind_df)

firewheel_wind_df.to_csv('output/firewheel_wind_turbines.csv', index=False)

So the wind project in Texas with the most turbines is Firewheel Wind out in Hansford County. We need to make sure that these are all operational — or maybe not. Maybe its OK that these are just planned.

Let's find the first record for each project so that we can map out specific locations.

In [8]:
# For each wind farm, only keep the first row. This will remove duplicate wind farms.
single_point_df = texas_df.drop_duplicates(subset='p_name', keep='first')

# Filter out "unknown" wind farms
single_point_df = single_point_df[~single_point_df['p_name'].str.contains('unknown', case=False)]

show(single_point_df)

single_point_df.to_csv('output/texas_wind_farm_tnum.csv', index=False)